In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.interpolate import griddata

In [2]:
# general mesh info
file_14 = r'./hgrid.gr3'
mesh_info = []
with open(file_14, 'r', encoding='utf-8') as f:
    for i in range(0,2):
        wz = f.readline()
        wz = wz.strip('\n')
        wz = wz.split(' ')
        print(wz)
        mesh_info.append(wz)
f.close()
mesh_info
ncell = np.int64(mesh_info[1][0])
npoint = np.int64(mesh_info[1][1])
print(ncell, npoint)

['OceanMesh2D', '']
['24462', '13561', '']
24462 13561


In [3]:
points = pd.read_csv(file_14, skiprows=2, header=None, nrows=npoint, sep='\s+')

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_42263/908926587.py:1: SyntaxWarning: invalid escape sequence '\s'
  points = pd.read_csv(file_14, skiprows=2, header=None, nrows=npoint, sep='\s+')


In [4]:
points.columns = ['id', 'lon', 'lat', 'depth']

In [16]:
import os
os.system("rm ./atmos.nc")
save_file = r'./atmos.nc'

date_ran = pd.date_range("2019-10-29 23", "2019-11-03 00", freq="1h")
time_count = len(date_ran)
ds = xr.open_dataset(r'./out.nc')

lon = ds.longitude
lat = ds.latitude
lons,lats = np.meshgrid(lon, lat)
grid_in = np.concatenate([lons.reshape(-1,1), lats.reshape(-1,1)], axis=1)

grid_out = np.vstack((points['lon'], points['lat'])).T

u10_out = np.empty((time_count,npoint))
v10_out = np.empty((time_count,npoint))
sp_out = np.empty((time_count,npoint))

# interpolation
outputs =  np.empty((0,3))
for itime in range(time_count):
    u10_in = ds.u10.sel(valid_time=date_ran[itime], method='nearest')
    v10_in = ds.v10.sel(valid_time=date_ran[itime], method='nearest')
    sp_in = ds.sp.sel(valid_time=date_ran[itime], method='nearest')
    u10_out[itime] = griddata(grid_in, np.array(u10_in).flatten(), grid_out, method='linear')
    v10_out[itime] = griddata(grid_in, np.array(v10_in).flatten(), grid_out, method='linear')
    sp_out[itime] = griddata(grid_in, np.array(sp_in).flatten(), grid_out, method='linear')
#     output = np.vstack((points['id'],u10_out,v10_out,sp_out)).T
#     outputs = np.concatenate([outputs, output])
    print(str(itime)+': '+str(date_ran[itime]))


rm: cannot remove './atmos.nc': No such file or directory


0: 2019-10-29 23:00:00
1: 2019-10-30 00:00:00
2: 2019-10-30 01:00:00
3: 2019-10-30 02:00:00
4: 2019-10-30 03:00:00
5: 2019-10-30 04:00:00
6: 2019-10-30 05:00:00
7: 2019-10-30 06:00:00
8: 2019-10-30 07:00:00
9: 2019-10-30 08:00:00
10: 2019-10-30 09:00:00
11: 2019-10-30 10:00:00
12: 2019-10-30 11:00:00
13: 2019-10-30 12:00:00
14: 2019-10-30 13:00:00
15: 2019-10-30 14:00:00
16: 2019-10-30 15:00:00
17: 2019-10-30 16:00:00
18: 2019-10-30 17:00:00
19: 2019-10-30 18:00:00
20: 2019-10-30 19:00:00
21: 2019-10-30 20:00:00
22: 2019-10-30 21:00:00
23: 2019-10-30 22:00:00
24: 2019-10-30 23:00:00
25: 2019-10-31 00:00:00
26: 2019-10-31 01:00:00
27: 2019-10-31 02:00:00
28: 2019-10-31 03:00:00
29: 2019-10-31 04:00:00
30: 2019-10-31 05:00:00
31: 2019-10-31 06:00:00
32: 2019-10-31 07:00:00
33: 2019-10-31 08:00:00
34: 2019-10-31 09:00:00
35: 2019-10-31 10:00:00
36: 2019-10-31 11:00:00
37: 2019-10-31 12:00:00
38: 2019-10-31 13:00:00
39: 2019-10-31 14:00:00
40: 2019-10-31 15:00:00
41: 2019-10-31 16:00:00
42

In [14]:
print(u10_in.shape)
print(grid_out.shape)

(26, 51)
(13561, 2)


In [23]:
# create NetCDF file
ds1 = xr.Dataset(
    data_vars={
        "uwind": (("time", "node"), u10_out),
        "vwind": (("time", "node"), v10_out),
        "prmsl": (("time", "node"), sp_out),
    },
    coords={
        "time": np.arange(time_count),
        "node": np.arange(npoint),
    },
)

In [24]:
print ('saving to ', save_file)
ds1.to_netcdf(path=save_file)

saving to  ./atmos.nc
